<a href="https://colab.research.google.com/github/lockiultra/greenatom/blob/main/status_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
!pip install -q evaluate
!pip install -U accelerate
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 4.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import evaluate
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, BertTokenizer
from datasets.arrow_dataset import Dataset

import warnings
warnings.filterwarnings('ignore')

In [4]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
rating_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=10)
status_model = AutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
def tokenize(examples):
  return tokenizer(examples['text'], truncation=True, padding=True)

In [6]:
metric = evaluate.load('accuracy')

def compute_metric(eval_pred):
  logits, labels = eval_pred
  preds = np.argmax(logits, axis=-1)
  return metric.compute(predictions=preds, references=labels)

In [7]:
status_training_args = TrainingArguments(output_dir='test_trainer', evaluation_strategy='epoch')

In [8]:
df_train = pd.read_csv('/content/drive/MyDrive/greenatom/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/greenatom/test.csv')

df_status_train = df_train[['text', 'status']]
df_status_train.columns = ['text', 'label']
df_status_test = df_test[['text', 'status']]
df_status_test.columns = ['text', 'label']

In [9]:
train_status_dataset = Dataset.from_pandas(df_status_train).map(tokenize, batched=True)
test_status_dataset = Dataset.from_pandas(df_status_test).map(tokenize, batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

In [10]:
torch.cuda.empty_cache()

In [11]:
status_trainer = Trainer(
    model=status_model,
    args=status_training_args,
    train_dataset=train_status_dataset,
    eval_dataset=test_status_dataset,
    compute_metrics=compute_metric
)

In [12]:
status_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.334600,0.329472,0.909000
2,0.226800,0.323987,0.918080
3,0.154000,0.329861,0.926800


TrainOutput(global_step=9375, training_loss=0.25298737955729167, metrics={'train_runtime': 9515.3835, 'train_samples_per_second': 7.882, 'train_steps_per_second': 0.985, 'total_flos': 1.9733329152e+16, 'train_loss': 0.25298737955729167, 'epoch': 3.0})

In [14]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cr

In [15]:
status_trainer.model.push_to_hub('status_model')

pytorch_model.bin:   0%|          | 0.00/433M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/lockiultra/status_model/commit/4836b575151fb26d8cface848bec3880adced1cc', commit_message='Upload BertForSequenceClassification', commit_description='', oid='4836b575151fb26d8cface848bec3880adced1cc', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
tokenizer.push_to_hub('status_model')

CommitInfo(commit_url='https://huggingface.co/lockiultra/status_model/commit/99a507ed0d3dbd79eb4319e741235182516da3ba', commit_message='Upload tokenizer', commit_description='', oid='99a507ed0d3dbd79eb4319e741235182516da3ba', pr_url=None, pr_revision=None, pr_num=None)